<a href="https://colab.research.google.com/github/sriramdussa/fmml-jan-2022/blob/main/Lab14_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FOUNDATIONS OF MODERN MACHINE LEARNING, IIIT Hyderabad
## Lecture 14 Project
### Lab Coordinator: Shantanu Agrawal

In [1]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Importing required libraries and packages
import pandas as pd
import tqdm

from sklearn.preprocessing import LabelEncoder

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

from google.colab import files

In [3]:
# Now we import the Dataset module to inherit various functions such as __getitem__(), __len__(), etc predefined in the library. 
class TitanicDataset(Dataset):
  def __init__(self,csvpath, mode = 'train'):
    self.mode = mode
    df = pd.read_csv(csvpath)
    le = LabelEncoder()        
    """
    <------Some Data Preprocessing---------->
    Removing Null Values, Outliers and Encoding the categorical labels etc

    Also, look at the difference between the train or test modes
    """
    if self.mode == 'train':
      # df = df.dropna()
      self.inp = df.iloc[:,2:]
      for i in range(self.inp.shape[1]):
        le = LabelEncoder()
        le.fit(self.inp.iloc[:,i])
        self.inp.iloc[:,i] = le.transform(self.inp.iloc[:,i])
      self.inp = self.inp.values
      self.oup = df.iloc[:,1].values.reshape(len(self.inp),1)
    else:
      self.inp = df.iloc[:,1:]
      for i in range(self.inp.shape[1]):
        le = LabelEncoder()
        le.fit(self.inp.iloc[:,i])
        self.inp.iloc[:,i] = le.transform(self.inp.iloc[:,i])
      self.inp = self.inp.values
    
  def __len__(self):
    return len(self.inp)
    
  def __getitem__(self, idx):
    """
      Look at how result has been returned in the form of dictionary 
      for easier understanding as well as easier accessing.
    """
    if self.mode == 'train':
      inpt  = torch.Tensor(self.inp[idx])
      oupt  = torch.Tensor(self.oup[idx])
      return { 'inp': inpt,
              'oup': oupt,
          }
    else:
      inpt = torch.Tensor(self.inp[idx])
      return { 'inp': inpt
      }

In [4]:
# Use this to import train.csv file given to you
files.upload()

Saving train.csv to train.csv


{'train.csv': b'PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked\r\n1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S\r\n2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C\r\n3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S\r\n4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S\r\n5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S\r\n6,0,3,"Moran, Mr. James",male,,0,0,330877,8.4583,,Q\r\n7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S\r\n8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,,S\r\n9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S\r\n10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,,C\r\n11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,PP 9549,16.7,G6,S\r\n12,1,1,"Bonnell, Miss. Elizabeth",female,58,0,0,113783,26.55,C103,S\r\

In [5]:
 data = TitanicDataset('/content/train.csv')

In [6]:
## Initialize the Training DataSet
data = TitanicDataset('train.csv')

## --- TASK 1 ---
## Choose batch size of your own for the data loader
## Fill in place of "??"
BATCH_SIZE = 56112

## DataLoader has been initialized as below
## Look for the use of this
data_train = DataLoader(dataset = data, batch_size = BATCH_SIZE, shuffle =False)

**NOTE:** You have to pass your dataset object resulting from the previous function as your argument. According to the number of batches, the result will be a multidimensional tensor of the shape **(no_of_batches, batch_size, size_of_the_vector)**.

It's the time for model generation, you will be creating mutiple perceptron layers to get the final model.

**---- TASK 2 ----**<br>
This is your task to design your layers yourselves.<br>
Makes sure:
- Input length of the first should be equal to total number of features.
- Output length of final layer should be equal to 1.
- You are using non-linearitites (i.e, activation layers) in between the layers to ensure the property.
- Look for BatchNorm1D layer as well, for better results.

You can examine the dataset yourself by reading the csv using the panda library (via pd.read_csv() function) to explore more about it.

In [7]:
class Network(nn.Module):

    def __init__(self):
        super().__init__()
        #define various layers
        self.fc1 = nn.Linear(10, 8, bias=True) 
        self.fc2 = nn.Linear(8, 6, bias=True)
        self.fc3 = nn.Linear(6, 4, bias=True)
        self.fc4 = nn.Linear(4, 2, bias=True)
        self.fc5 = nn.Linear(2, 1, bias=True)

    def swish(self, x):
        return x * F.sigmoid(x)

    def forward(self,x):
        # use this swish function or F.relu() in place of that
        x = self.fc1(x)
        x = self.swish(x)
        x = self.fc2(x)
        x = self.swish(x)
        x = self.fc3(x)
        x = self.swish(x)
        x = self.fc4(x)
        x = self.swish(x)
        x = self.fc5(x)
        x = self.swish(x)
        return x

In [8]:
# Training paradigm has been setup for you here
def train(model, x, y, optimizer, criterion):
    model.zero_grad()
    output = model(x)
    loss =criterion(output,y)
    loss.backward()
    optimizer.step()

    return loss, output

In [9]:
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("Cuda Device Available")
  print("Name of the Cuda Device: ", torch.cuda.get_device_name())
  print("GPU Computational Capablity: ", torch.cuda.get_device_capability())

In [10]:
# ---- TASK 3 ----
# Edit the number of epochs (more than once (and then run the code)) to check the result(s)
# Fill in place of "??"
EPOCHS = 3000

net = Network()
optm = Adam(net.parameters(), lr = 0.001)
data_train = DataLoader(dataset = data, batch_size = BATCH_SIZE, shuffle =False)
criterion = nn.MSELoss()

for epoch in range(EPOCHS):
    epoch_loss = 0
    correct = 0
    for batch in data_train:
        x_train, y_train = batch['inp'], batch['oup']
        x_train = x_train.view(-1,10)
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        loss, predictions = train(net,x_train,y_train, optm, criterion)
        for idx, i in enumerate(predictions):
            i  = torch.round(i)
            if i == y_train[idx]:
                correct += 1
        acc = (correct/len(data))
        epoch_loss+=loss
    print('Epoch {} Accuracy : {}'.format(epoch+1, acc*100))
    print('Epoch {} Loss : {}'.format((epoch+1),epoch_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Streaming output truncated to the last 5000 lines.
Epoch 501 Accuracy : 68.91133557800224
Epoch 501 Loss : 0.20241481065750122
Epoch 502 Accuracy : 69.02356902356902
Epoch 502 Loss : 0.20232424139976501
Epoch 503 Accuracy : 69.24803591470258
Epoch 503 Loss : 0.20223335921764374
Epoch 504 Accuracy : 69.1358024691358
Epoch 504 Loss : 0.2021421492099762
Epoch 505 Accuracy : 69.1358024691358
Epoch 505 Loss : 0.2020505964756012
Epoch 506 Accuracy : 69.02356902356902
Epoch 506 Loss : 0.20195871591567993
Epoch 507 Accuracy : 69.02356902356902
Epoch 507 Loss : 0.20186646282672882
Epoch 508 Accuracy : 69.02356902356902
Epoch 508 Loss : 0.20177382230758667
Epoch 509 Accuracy : 69.1358024691358
Epoch 509 Loss : 0.20168070495128632
Epoch 510 Accuracy : 69.1358024691358
Epoch 510 Loss : 0.20158708095550537
Epoch 511 Accuracy : 69.1358024691358
Epoch 511 Loss : 0.20149284601211548
Epoch 512 Accuracy : 68.91133557800224
Epoch 512 Loss : 0.20139798521995544
Epoch 513 Accuracy : 69.24803591470258
Epoch

Above shown results are only **training loss and accuracy**. There are various factors for this 100% result.<br>
You should also look for **test/validation loss** as well for better understanding of how model is working.

In [11]:
files.upload()

Saving test.csv to test.csv


{'test.csv': b'PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked\r\n892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,,Q\r\n893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7,,S\r\n894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,,Q\r\n895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,,S\r\n896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22,1,1,3101298,12.2875,,S\r\n897,3,"Svensson, Mr. Johan Cervin",male,14,0,0,7538,9.225,,S\r\n898,3,"Connolly, Miss. Kate",female,30,0,0,330972,7.6292,,Q\r\n899,2,"Caldwell, Mr. Albert Francis",male,26,1,1,248738,29,,S\r\n900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18,0,0,2657,7.2292,,C\r\n901,3,"Davies, Mr. John Samuel",male,21,2,0,A/4 48871,24.15,,S\r\n902,3,"Ilieff, Mr. Ylio",male,,0,0,349220,7.8958,,S\r\n903,1,"Jones, Mr. Charles Cresson",male,46,0,0,694,26,,S\r\n904,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23,1,0,21228,82.2667,B45,S\r\n905,2,"Howard, Mr. Benja

In [12]:
data = TitanicDataset('/content/test.csv')

In [13]:
BATCH_SIZE = 13
data_test = DataLoader(dataset = data, batch_size = BATCH_SIZE, shuffle =False)

In [14]:
class Network(nn.Module):

    def __init__(self):
        super().__init__()
        #define various layers
        self.fc1 = nn.Linear(9, 8, bias=True) 
        self.fc2 = nn.Linear(8, 6, bias=True)
        self.fc3 = nn.Linear(6, 4, bias=True)
        self.fc4 = nn.Linear(4, 2, bias=True)
        self.fc5 = nn.Linear(2, 1, bias=True)

    def swish(self, x):
        return x * F.sigmoid(x)

    def forward(self,x):
        # use this swish function or F.relu() in place of that
        x = self.fc1(x)
        x = self.swish(x)
        x = self.fc2(x)
        x = self.swish(x)
        x = self.fc3(x)
        x = self.swish(x)
        x = self.fc4(x)
        x = self.swish(x)
        x = self.fc5(x)
        x = self.swish(x)
        return x

In [15]:
def test(model, x, y, optimizer, criterion):
    model.zero_grad()
    output = model(x)
    loss =criterion(output,y)
    loss.backward()
    optimizer.step()

    return loss, output

In [16]:
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("Cuda Device Available")
  print("Name of the Cuda Device: ", torch.cuda.get_device_name())
  print("GPU Computational Capablity: ", torch.cuda.get_device_capability())

In [17]:
EPOCHS = 3000

net = Network()
optm = Adam(net.parameters(), lr = 0.001)
data_test = DataLoader(dataset = data, batch_size = BATCH_SIZE, shuffle =False)
criterion = nn.MSELoss()

for epoch in range(EPOCHS):
    epoch_loss = 0
    correct = 0
    for batch in data_test:
        x_test, y_test = batch['inp'], batch['oup']
        x_test = x_test.view(-1,9)
        x_test = x_test.to(device)
        y_test = y_test.to(device)
        loss, predictions = test(net,x_test,y_test, optm, criterion)
        for idx, i in enumerate(predictions):
            i  = torch.round(i)
            if i == y_test[idx]:
                correct += 1
        acc = (correct/len(data))
        epoch_loss+=loss

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [18]:
print('Accuracy : {}'.format(acc*100))
print('Loss : {}'.format(epoch_loss))

Accuracy : 94.73684210526315
Loss : 1.8492940664291382
